In [1]:
import os
import re
from os import PathLike
from pathlib import Path
from typing import List, Optional, Sequence, Union

import numpy as np
import pandas as pd
import tifffile



In [2]:
import logging

logger = logging.getLogger(__name__.rpartition(".")[0])


class SteinbockException(Exception):
    pass



In [3]:


def _fix_mask_shape(mask_file: Union[str, PathLike], mask: np.ndarray) -> np.ndarray:
    if mask.ndim == 3:
        if mask.shape[0] == 1:
            mask = mask[0, :, :]
        elif mask.shape[-1] == 1:
            mask = mask[:, :, 0]
        else:
            raise SteinbockIOException(
                f"{mask_file}: unsupported three-dimensional shape {mask.shape}"
            )
    elif mask.ndim == 4:
        if mask.shape[0] == 1 and mask.shape[1] == 1:
            mask = mask[0, 0, :, :]
        elif mask.shape[0] == 1 and mask.shape[-1] == 1:
            mask = mask[0, :, :, 0]
        elif mask.shape[-1] == 1 and mask.shape[-2] == 1:
            mask = mask[:, :, 0, 0]
        else:
            raise SteinbockIOException(
                f"{mask_file}: unsupported four-dimensional shape {mask.shape}"
            )
    elif mask.ndim == 5:
        size_t, size_z, size_c, size_y, size_x = mask.shape
        if size_t != 1 or size_z != 1 or size_c != 1:
            raise SteinbockIOException(
                f"{mask_file}: unsupported TZCYX shape {mask.shape}"
            )
        mask = mask[0, 0, 0, :, :]
    elif mask.ndim == 6:
        size_t, size_z, size_c, size_y, size_x, size_s = mask.shape
        if size_t != 1 or size_z != 1 or size_c != 1 or size_s != 1:
            raise SteinbockIOException(
                f"{mask_file}: unsupported TZCYXS shape {mask.shape}"
            )
        mask = mask[0, 0, 0, :, :, 0]
    elif mask.ndim != 2:
        raise SteinbockIOException(
            f"{mask_file}: unsupported number of dimensions ({mask.ndim})"
        )
    return mask

In [4]:
def read_mask(
    mask_file: Union[str, PathLike],
    native_dtype: bool = False,
) -> np.ndarray:
    mask = tifffile.imread(mask_file, squeeze=False)
    mask = _fix_mask_shape(mask_file, mask)
    if not native_dtype:
        mask = _to_dtype(mask, mask_dtype)
    return mask



In [22]:
# folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/masks_plaques/" #'/Users/eleonoreschneegans/Desktop/test_imc/masks_plaques'

# os.chdir(folder_path)
# os.getcwd()

'/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/masks_plaques'

In [5]:
# folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/new_masks/"
# folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/masks_plaques_ori/masks_plaques/"
folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/new_plaques_global/"
os.chdir(folder_path)
os.getcwd()

'/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/new_plaques_global'

In [6]:
import glob
# Create a list of .tiff file paths
# mask_files = glob.glob(folder_path + "/*.tiff")
# Alternatively, create a tuple of .tiff file paths
# mask_files = tuple(glob.glob(folder_path + "/*.tiff"))
# Get a list of .tiff file paths
mask_files = glob.glob("*.tiff")
mask_files

['DPM16.44_002.tiff',
 'A19.12_002.tiff',
 'NP14.128_002.tiff',
 '19930222_003.tiff',
 '19920194_004.tiff',
 'A319.11_002.tiff',
 'A115.17_003.tiff',
 '19891053_004.tiff',
 'PDC26_003.tiff',
 '19930148_004.tiff',
 'A127.11_001.tiff',
 'A53.11_003.tiff',
 'A163.17_002.tiff',
 'P23.07_002.tiff',
 '19920077_001.tiff',
 '19910006_004.tiff',
 'NP13.012_003.tiff',
 '19920035_004.tiff',
 '19930154_004.tiff',
 '19920077_003.tiff',
 'A13.03_002.tiff',
 'A169.10_002.tiff',
 'A282.14_004.tiff',
 'A127.11_003.tiff',
 'A220.11_003.tiff',
 '20185935_004.tiff',
 'A282.14_001.tiff',
 '20110137_001.tiff',
 'A170.13_002.tiff',
 'A282.14_002.tiff',
 '19910006_003.tiff',
 '20110137_004.tiff',
 '19891053_003.tiff',
 'A163.17_004.tiff',
 'A51.14_002.tiff',
 'A138.12_001.tiff',
 'NP14.128_001.tiff',
 '20130325_004.tiff',
 'A297.16_001.tiff',
 '962.2_001.tiff',
 'A13.03_004.tiff',
 '19960064_001.tiff',
 '19930154_002.tiff',
 'A96.14_002.tiff',
 '20185935_003.tiff',
 'PDC16_001.tiff',
 'A170.13_004.tiff',
 '19

In [11]:
# mask_file='/rds/general/user/wc1322/home/summer-project/combined_data/10_mtg/masks/DPM16.44_002.tiff'

In [12]:
# mask_2 = read_mask(mask_file, native_dtype=True)



In [13]:
# mask_2

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   2, ...,   0,   0,   0],
       [  0,   0,   2, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 597,   0,   0],
       [  0,   0,   0, ..., 597,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint16)

In [7]:
import numpy as np
from skimage.segmentation import expand_labels

import io


def expand_mask(mask: np.ndarray, distance: int) -> np.ndarray:
    expanded_mask = expand_labels(mask, distance=distance)
    return expanded_mask



In [15]:
# expanded_mask = expand_mask(mask_2, distance=10)

In [16]:
# expanded_mask

array([[  2,   2,   2, ...,   0,   0,   0],
       [  2,   2,   2, ...,   0,   0,   0],
       [  2,   2,   2, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 597, 597, 597],
       [  0,   0,   0, ..., 597, 597, 597],
       [  0,   0,   0, ..., 597, 597, 597]], dtype=uint16)

In [9]:
def write_image(
    img: np.ndarray,
    img_file: Union[str, PathLike],
    ignore_dtype: bool = False,
) -> None:
    if not ignore_dtype:
        img = _to_dtype(img, img_dtype)
    tifffile.imwrite(
        img_file,
        data=img,
        imagej=img.dtype in (np.uint8, np.uint16, np.float32),
    )

def _to_dtype(src: np.ndarray, dst_dtype: np.dtype) -> np.ndarray:
    if src.dtype == dst_dtype:
        return src
    src_is_int = np.issubdtype(src.dtype, np.integer)
    dst_is_int = np.issubdtype(dst_dtype, np.integer)
    if not src_is_int and dst_is_int:
        src = np.around(src)
    if src_is_int:
        src_info = np.iinfo(src.dtype)
    else:
        src_info = np.finfo(src.dtype)
    if dst_is_int:
        dst_info = np.iinfo(dst_dtype)
    else:
        dst_info = np.finfo(dst_dtype)
    if src_info.min < dst_info.min or src_info.max > dst_info.max:
        src = np.clip(src, dst_info.min, dst_info.max)
    return src.astype(dst_dtype)




In [18]:
# expanded_mask

array([[  2,   2,   2, ...,   0,   0,   0],
       [  2,   2,   2, ...,   0,   0,   0],
       [  2,   2,   2, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 597, 597, 597],
       [  0,   0,   0, ..., 597, 597, 597],
       [  0,   0,   0, ..., 597, 597, 597]], dtype=uint16)

In [19]:
# write_image(expanded_mask,img_file='/rds/general/user/wc1322/home/summer-project/combined_data/10_mtg/masks/DPM16.44_002_NEW.tiff',ignore_dtype=True)

In [10]:
## Expand the masks_plaques by 50
# new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/masks_plaques_5/"
# new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/new_masks_50/"
new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/new_plaques_global_50/"
# Create the new folder if it doesn't exist
os.makedirs(new_folder_path, exist_ok=True)

# Iterate over the mask files
for mask_file in mask_files:
    # Read the mask file
    mask = read_mask(mask_file, native_dtype=True)
    
    # Expand the mask
    expanded_mask = expand_mask(mask, distance=50)
    
    # Create the new mask file path in the new folder
    new_mask_file = os.path.join(new_folder_path, os.path.basename(mask_file))
    
    # Write the expanded mask to the new file
    write_image(expanded_mask, img_file=new_mask_file, ignore_dtype=True)

In [11]:
## Expand the masks_plaques by 3
# new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/masks_plaques_30/"
new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/new_plaques_global_30/"
# Create the new folder if it doesn't exist
os.makedirs(new_folder_path, exist_ok=True)

# Iterate over the mask files
for mask_file in mask_files:
    # Read the mask file
    mask = read_mask(mask_file, native_dtype=True)
    
    # Expand the mask
    expanded_mask = expand_mask(mask, distance=30)
    
    # Create the new mask file path in the new folder
    new_mask_file = os.path.join(new_folder_path, os.path.basename(mask_file))
    
    # Write the expanded mask to the new file
    write_image(expanded_mask, img_file=new_mask_file, ignore_dtype=True)

In [12]:
## Expand the masks_plaques by 10
# new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/masks_plaques_10/"
new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/new_plaques_global_10/"

# Create the new folder if it doesn't exist
os.makedirs(new_folder_path, exist_ok=True)

# Iterate over the mask files
for mask_file in mask_files:
    # Read the mask file
    mask = read_mask(mask_file, native_dtype=True)
    
    # Expand the mask
    expanded_mask = expand_mask(mask, distance=10)
    
    # Create the new mask file path in the new folder
    new_mask_file = os.path.join(new_folder_path, os.path.basename(mask_file))
    
    # Write the expanded mask to the new file
    write_image(expanded_mask, img_file=new_mask_file, ignore_dtype=True)

In [13]:
## Expand the masks_plaques by 10
# new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/masks_plaques_20/"
new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/new_plaques_global_20/"

# Create the new folder if it doesn't exist
os.makedirs(new_folder_path, exist_ok=True)

# Iterate over the mask files
for mask_file in mask_files:
    # Read the mask file
    mask = read_mask(mask_file, native_dtype=True)
    
    # Expand the mask
    expanded_mask = expand_mask(mask, distance=20)
    
    # Create the new mask file path in the new folder
    new_mask_file = os.path.join(new_folder_path, os.path.basename(mask_file))
    
    # Write the expanded mask to the new file
    write_image(expanded_mask, img_file=new_mask_file, ignore_dtype=True)

In [14]:
## Expand the masks_plaques by 10
# new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/masks_plaques_40/"
new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/new_plaques_global_40/"
# Create the new folder if it doesn't exist
os.makedirs(new_folder_path, exist_ok=True)

# Iterate over the mask files
for mask_file in mask_files:
    # Read the mask file
    mask = read_mask(mask_file, native_dtype=True)
    
    # Expand the mask
    expanded_mask = expand_mask(mask, distance=40)
    
    # Create the new mask file path in the new folder
    new_mask_file = os.path.join(new_folder_path, os.path.basename(mask_file))
    
    # Write the expanded mask to the new file
    write_image(expanded_mask, img_file=new_mask_file, ignore_dtype=True)

In [15]:
## Expand the masks_plaques by 100
# new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/masks_plaques_40/"
# new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/new_masks_100/"
# new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/masks_plaques_ori/masks_plaques_100/"
new_folder_path = "/rds/general/user/wc1322/home/summer-project/combined_data/mtg_dna/deepcell/new_plaques_global_100/"

# Create the new folder if it doesn't exist
os.makedirs(new_folder_path, exist_ok=True)

# Iterate over the mask files
for mask_file in mask_files:
    # Read the mask file
    mask = read_mask(mask_file, native_dtype=True)
    
    # Expand the mask
    expanded_mask = expand_mask(mask, distance=100)
    
    # Create the new mask file path in the new folder
    new_mask_file = os.path.join(new_folder_path, os.path.basename(mask_file))
    
    # Write the expanded mask to the new file
    write_image(expanded_mask, img_file=new_mask_file, ignore_dtype=True)